In [1]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from IPython.display import display, Markdown
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
# !pip install pypdf

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Robinson_Advisory.pdf")
pages = loader.load_and_split()

In [5]:
pages[0]

Document(page_content='ADVISORY SERVICES AGREEMENT  \n \nThis Advisory Services Agreement is entered into as of June  15th, 2023 (the “ Effective Date”), by and between \nCloud Investments Ltd., ID 51 -426526- 3, an Israeli company (the "Company "), and Mr. Jack Robinson, \nPassport Number 780055578, residing at 1 Rabin st, Tel Aviv, Israel, Email: jackrobinson@gmail.com \n("Advisor "). \n \nWhereas , Advisor ha s expertise and/or knowledge and/or relationships, which are relevant to the \nCompany’s business and the Company has asked Advisor to provide it with certain Advisory \nservices, as described in this Agreement; and  \nWhereas ,  Advisor has agreed to provide the Company with such services, subject to the terms s et forth \nin this Agreement.  \n NOW THEREFORE THE PARTIES AGREE AS FOLLOWS:  \n \n1. Services:    \n1.1 Advisor shall provide to the Company, as an independent contractor, software development  \nservices,  and / or any other services as agreed by the parties from ti

In [6]:
from langchain.text_splitter import SpacyTextSplitter
from langchain.text_splitter import CharacterTextSplitter

with open("../data/Robinson_Advisory.txt", encoding="windows-1252") as f:
    document = f.read()

In [7]:
from langchain.text_splitter import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=1000)

In [8]:

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=0
)
documents = loader.load_and_split()
# texts = text_splitter.split_text(document)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [9]:
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    collection_name="contract",
    connection_args={"host": "127.0.0.1", "port": "19530", "database": "contract"},
)

In [10]:
vector_db = Milvus(
    embeddings,
    connection_args={"host": "127.0.0.1", "port": "19530"},
    collection_name="contract",
)

In [11]:
query = "Who are the parties to the Agreement and what are their defined names?"
docs = vector_db.similarity_search(query)

In [12]:
docs[0].page_content

'4. Term: The term of this Agreement shall commence on the Effective Date and shall continue until terminated in accordance with the provisions herein (the "Term").  \n\n5. Termination: Either party, at any given time, may terminate this Agreement, for any reason whatsoever, with or without cause, upon fourteen (14) days’ prior written notice. Notwithstanding the above, the Company may terminate this Agreement immediately and without prior notice if Advisor refuses or is unable to perform the Services, or is in breach of any provision of this Agreement.'

In [13]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
query = "What is the termination notice?"
docs = vector_db.similarity_search(query)

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [14]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [15]:
response = llm.call_as_llm(f"{qdocs} Question: {query}") 
display(Markdown(response))

/root/Contract-Advisor-RAG-/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `call_as_llm` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The termination notice is fourteen (14) days' prior written notice.

In [40]:
data = loader.load()
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_db.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [41]:
data[1]

Document(page_content='-  2- \n \n4. Term : The term of this Agreement shall commence on the Effective Date and shall continue until \nterminated in accordance with the provisions herein (the " Term ").   \n \n5. Term ination : Either party, at any given time, may terminate this Agreement, for any reason \nwhatsoever, with or without cause, upon fourteen  (14) days’ prior written notice . Notwithstanding the \nabove, the Company may terminate this Agreement immediately and without prior notice if Advisor \nrefuses or is unable to perform the Services, or is in breach of any provision of this Agreement.  \n \n6. Compensation: \n6.1 As full and sole consideration for the Services, and subject to the performance of the Services, the \nCompany shall pay the Advisor , against an invoice provided to the Company by Advisor, hourly \nfees at a rate of USD 9 (nine) per Billable Hour as defined below, limited to a maximum of USD \n1,500 per month (the " Fees "). In addition, the Company shall pa

In [42]:
examples = [
    {
        "query": "Who are the parties to the Agreement and what are their defined names?",
        "answer": "Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)"
    },
    {
        "query": "What is the termination notice?",
        "answer": "According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement."
    },
    {
        "query": "What are the payments to the Advisor under the Agreement?",
        "answer": "According to section 6: 1. Fees of $9 per hour up to a monthly limit of $1,500, 2. Workspace expense of $100 per month, 3. Other reasonable and actual expenses if approved by the company in writing and in advance."
    },
    {
        "query": "Can the Agreement or any of its obligations be assigned?",
        "answer": "Under section 1.1 the Advisor can’t assign any of his obligations without the prior written consent of the Company, 2. Under section 9 the Advisor may not assign the Agreement and the Company may assign it, 3 Under section 9 of the Undertaking the Company may assign the Undertaking."
    },
    {
        "query": "Who owns the IP?",
        "answer": "According to section 4 of the Undertaking (Appendix A), Any Work Product, upon creation, shall be fully and exclusively owned by the Company."
    },
    {
        "query": " Can the Advisor charge for meal time?",
        "answer": "No. See Section 6.1, Billable Hour doesn’t include meals or travel time."
    },
    {
        "query": " Is there a non-compete obligation to the Advisor?",
        "answer": "Yes. During the term of engagement with the Company and for a period of 12 months thereafter."
    },
    {
        "query": "In which street does the Advisor live?",
        "answer": "1 Rabin st, Tel Aviv, Israel"
    },
    {
        "query": "Is the Advisor entitled to social benefits?",
        "answer": "No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company."
    },
    {
        "query": "What happens if the Advisor claims compensation based on employment relationship with the Company?",
        "answer": "If the Advisor is determined to be an employee of the Company by a governmental authority, payments to the Advisor will be retroactively reduced so that 60% constitutes salary payments and 40% constitutes payment for statutory rights and benefits. The Company may offset any amounts due to the Advisor from any amounts payable under the Agreement. The Advisor must indemnify the Company for any losses or expenses incurred if an employer/employee relationship is determined to exist."
    },
]

In [43]:
import langchain
langchain.debug = False

In [44]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...



> Finished chain.


'The context provided does not include information about the parties to the Agreement or their defined names.'

In [45]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [46]:
from langchain.evaluation.qa import QAEvalChain

In [47]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [48]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [51]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Who are the parties to the Agreement and what are their defined names?
Real Answer: Cloud Investments Ltd. (“Company”) and Jack Robinson (“Advisor”)
Predicted Answer: The context provided does not include information about the parties to the Agreement or their defined names.
Predicted Grade: INCORRECT

Example 1:
Question: What is the termination notice?
Real Answer: According to section 4:14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of this Agreement.
Predicted Answer: The termination notice is fourteen (14) days' prior written notice, which either party can give at any time, for any reason whatsoever, with or without cause, to terminate the Agreement. However, the Company may terminate the Agreement immediately and without prior notice if the Advisor refuses or is unable to perform the Services or is in breach of any provision of the Agreeme

In [49]:
length = len(graded_outputs)
print(f"The length of the list is: {length}")

for i in range(length):
    print(graded_outputs[i])

The length of the list is: 9
{'results': 'INCORRECT'}
{'results': 'CORRECT'}
{'results': 'CORRECT'}
{'results': 'CORRECT'}
{'results': 'CORRECT'}
{'results': 'CORRECT'}
{'results': 'CORRECT'}
{'results': 'CORRECT'}
{'results': 'CORRECT'}
